In [22]:
import collections

import folium
import geopy.distance
import numpy
import pandas
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

In [23]:
df_sitios_bog = pandas.read_csv("./data/sitios-de-interes-en-bogota.csv", sep=";")
df_sitios_bog = df_sitios_bog[["Nombre", "Geo Point"]]
df_sitios_bog["Lat"] = df_sitios_bog.apply(lambda row: float(row["Geo Point"].split(",")[0]), axis=1)
df_sitios_bog["Lng"] = df_sitios_bog.apply(lambda row: float(row["Geo Point"].split(",")[1]), axis=1)
df_sitios_bog.drop("Geo Point", axis=1, inplace=True)
df_sitios_bog = df_sitios_bog.head(101)
df_sitios_bog = df_sitios_bog.drop_duplicates(subset="Nombre")
df_sitios_bog = df_sitios_bog.set_index("Nombre")
df_sitios_bog

,Lat,Lng
Nombre,,
Escultura Nave Espacial,4.615251,-74.071534
Estatua Tomas Cipriano de Mosquera,4.597386,-74.076421
Estatua Ricardo Palma,4.602078,-74.066768
Monumento al Almirante José Prudencio Padilla,4.625450,-74.074206
Monumento 21 Ángeles,4.732816,-74.075095
...,...,...
Laboratorios Spaison Ltda,4.681562,-74.117520
Luminex Legrand,4.693044,-74.118457
Doña Juana ESP SA,4.499661,-74.143927


In [24]:
lat_mean = df_sitios_bog["Lat"].mean()
lng_mean = df_sitios_bog["Lng"].mean()

bogota_map = folium.Map(location=[lat_mean, lng_mean], zoom_start=12, tiles="cartodbdark_matter")
for idx, row in df_sitios_bog.iterrows():
    if idx == "Bavaria SA.":
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=5, color="yellow", fill=True, fill_color="white", fill_opacity=0.6, popup=f"{idx}").add_to(bogota_map)
    else:
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=5, color="blue", fill=True, fill_color="white", fill_opacity=0.8, popup=f"{idx}").add_to(bogota_map)

bogota_map

In [25]:
df_distancias = pandas.DataFrame()
for nodo1 in df_sitios_bog.index.to_list():
    for nodo2 in df_sitios_bog.index.to_list():
        df_distancias.loc[nodo1, nodo2] = round(10000 * geopy.distance.geodesic((df_sitios_bog.loc[nodo1, "Lat"], df_sitios_bog.loc[nodo1, "Lng"]), (df_sitios_bog.loc[nodo2, "Lat"], df_sitios_bog.loc[nodo2, "Lng"])).km)
df_distancias = df_distancias[df_distancias.columns].astype(int)
df_distancias.head()

,Escultura Nave Espacial,Estatua Tomas Cipriano de Mosquera,Estatua Ricardo Palma,Monumento al Almirante José Prudencio Padilla,Monumento 21 Ángeles,Museo del Oro,Instituto Caro y Cuervo,Palacio de San Carlos,Plaza Fundacional de Suba,Club Bellavista Colsubsidio,...,Bavaria Dirección de Ventas Bogotá,Bavaria SA.,BEG Fábrica de Lubricantes,Matadero Frigo Iberia,Protex SA.,Laboratorios Spaison Ltda,Luminex Legrand,Doña Juana ESP SA,Avianca,Notaría 14
Escultura Nave Espacial,0,20486,15497,11662,130066,14812,21444,21095,139701,234932,...,22399,73755,44194,144614,134627,89333,100553,150970,45639,29240
Estatua Tomas Cipriano de Mosquera,20486,0,11902,31131,149768,7063,3227,2298,159000,255326,...,33309,79558,23995,124138,145158,103653,115606,131492,59070,49187
Estatua Ricardo Palma,15497,11902,0,27131,144867,5677,10351,10695,154763,248518,...,35364,85294,34365,131995,148552,104386,115792,141979,60221,44229
Monumento al Almirante José Prudencio Padilla,11662,31131,27131,0,118732,26212,32568,32102,128200,224366,...,16993,67027,53953,154968,124967,78484,89429,159168,35774,18064
Monumento 21 Ángeles,130066,149768,144867,118732,0,144856,151300,150828,13258,110504,...,120835,118044,171351,272195,102749,73674,65183,268902,100243,100826


In [26]:
distance_matrix = df_distancias.values.tolist()
num_vehicles = 1
depot = 92

In [27]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(distance_matrix), num_vehicles, depot)

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return distance_matrix[from_node][to_node]


transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC

In [28]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [29]:
# Print solution on console.
if solution:
    print("Objective: {}".format(solution.ObjectiveValue() / 10000))
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    ruta = []
    while not routing.IsEnd(index):
        plan_output += " {} ->".format(manager.IndexToNode(index))
        ruta.append(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += " {}\n".format(manager.IndexToNode(index))
    ruta.append(manager.IndexToNode(index))
    print(plan_output)
else:
    print("No solution found!")

Objective: 161.4442
Route for vehicle 0:
 92 -> 49 -> 86 -> 30 -> 11 -> 93 -> 97 -> 12 -> 14 -> 20 -> 29 -> 40 -> 84 -> 68 -> 36 -> 48 -> 38 -> 57 -> 10 -> 44 -> 24 -> 83 -> 31 -> 55 -> 88 -> 37 -> 35 -> 81 -> 82 -> 90 -> 17 -> 61 -> 62 -> 98 -> 89 -> 91 -> 26 -> 46 -> 94 -> 15 -> 87 -> 67 -> 41 -> 96 -> 95 -> 32 -> 72 -> 71 -> 74 -> 66 -> 8 -> 4 -> 73 -> 80 -> 75 -> 9 -> 45 -> 25 -> 51 -> 13 -> 27 -> 78 -> 70 -> 42 -> 69 -> 53 -> 47 -> 23 -> 77 -> 54 -> 28 -> 76 -> 16 -> 60 -> 99 -> 79 -> 58 -> 21 -> 85 -> 19 -> 64 -> 3 -> 50 -> 59 -> 22 -> 34 -> 43 -> 0 -> 56 -> 2 -> 5 -> 6 -> 7 -> 1 -> 65 -> 33 -> 63 -> 52 -> 39 -> 18 -> 92



In [30]:
def get_bearing(p1, p2):
    long_diff = numpy.radians(p2.lon - p1.lon)
    lat1 = numpy.radians(p1.lat)
    lat2 = numpy.radians(p2.lat)
    x = numpy.sin(long_diff) * numpy.cos(lat2)
    y = numpy.cos(lat1) * numpy.sin(lat2) - (numpy.sin(lat1) * numpy.cos(lat2) * numpy.cos(long_diff))
    bearing = numpy.degrees(numpy.arctan2(x, y))
    if bearing < 0:
        return bearing + 360
    return bearing


def get_arrows(locations, color="black", size=4, n_arrows=3, weight_arrow=3):
    Point = collections.namedtuple("Point", field_names=["lat", "lon"])
    p1 = Point(locations[0][0], locations[0][1])
    p2 = Point(locations[1][0], locations[1][1])
    rotation = get_bearing(p1, p2) - 90
    arrow_lats = numpy.linspace(p1.lat, p2.lat, n_arrows + 2)[1 : n_arrows + 1]
    arrow_lons = numpy.linspace(p1.lon, p2.lon, n_arrows + 2)[1 : n_arrows + 1]
    arrows = []
    for points in zip(arrow_lats, arrow_lons):
        arrows.append(
            folium.RegularPolygonMarker(
                location=points,
                color=color,
                weight=weight_arrow,
                fill=True,
                fill_color=color,
                fill_opacity=1,
                number_of_sides=3,
                radius=size,
                rotation=rotation,
            )
        )
    return arrows

In [31]:
df_sitios_bog

,Lat,Lng
Nombre,,
Escultura Nave Espacial,4.615251,-74.071534
Estatua Tomas Cipriano de Mosquera,4.597386,-74.076421
Estatua Ricardo Palma,4.602078,-74.066768
Monumento al Almirante José Prudencio Padilla,4.625450,-74.074206
Monumento 21 Ángeles,4.732816,-74.075095
...,...,...
Laboratorios Spaison Ltda,4.681562,-74.117520
Luminex Legrand,4.693044,-74.118457
Doña Juana ESP SA,4.499661,-74.143927


In [39]:
lat_mean = df_sitios_bog["Lat"].mean()
lng_mean = df_sitios_bog["Lng"].mean()

bogota_map = folium.Map(location=[lat_mean, lng_mean], zoom_start=12, tiles="cartodbdark_matter")
for idx, row in df_sitios_bog.iterrows():
    if idx == "Bavaria SA.":
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=10, color="yellow", fill=True, fill_color="white", fill_opacity=0.6, popup=idx).add_to(bogota_map)
    else:
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=5, color="blue", fill=True, fill_color="white", fill_opacity=0.8, popup=idx).add_to(bogota_map)

for i in range(len(ruta) - 1):
    coordinates = [
        df_sitios_bog.reset_index().loc[ruta[i], ["Lat", "Lng"]].values.tolist(),
        df_sitios_bog.reset_index().loc[ruta[i + 1], ["Lat", "Lng"]].values.tolist(),
    ]

    color = "gray"
    weight = 2

    pl = folium.PolyLine(coordinates, color=color, weight=weight)
    bogota_map.add_child(pl)

    arrows = get_arrows(locations=coordinates, color=color, size=2, n_arrows=1)
    for arrow in arrows:
        arrow.add_to(bogota_map)


bogota_map

In [33]:
# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(len(distance_matrix), num_vehicles, depot)

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return distance_matrix[from_node][to_node]


transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.local_search_metaheuristic = routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
search_parameters.time_limit.seconds = 60
search_parameters.log_search = True

In [34]:
# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

In [35]:
# Print solution on console.
if solution:
    print("Objective: {}".format(solution.ObjectiveValue()/10000))
    index = routing.Start(0)
    plan_output = "Route for vehicle 0:\n"
    route_distance = 0
    ruta = []
    while not routing.IsEnd(index):
        plan_output += " {} ->".format(manager.IndexToNode(index))
        ruta.append(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += " {}\n".format(manager.IndexToNode(index))
    ruta.append(manager.IndexToNode(index))
    print(plan_output)
else:
    print("No solution found!")

Objective: 157.8616
Route for vehicle 0:
 92 -> 18 -> 39 -> 52 -> 63 -> 33 -> 65 -> 1 -> 7 -> 6 -> 5 -> 2 -> 56 -> 0 -> 43 -> 34 -> 22 -> 59 -> 50 -> 3 -> 64 -> 19 -> 85 -> 21 -> 58 -> 79 -> 28 -> 54 -> 47 -> 23 -> 77 -> 78 -> 27 -> 13 -> 51 -> 73 -> 25 -> 45 -> 80 -> 75 -> 9 -> 74 -> 66 -> 8 -> 4 -> 71 -> 72 -> 32 -> 70 -> 42 -> 69 -> 95 -> 96 -> 41 -> 67 -> 87 -> 15 -> 94 -> 46 -> 26 -> 91 -> 89 -> 98 -> 61 -> 62 -> 53 -> 16 -> 76 -> 99 -> 60 -> 17 -> 90 -> 82 -> 81 -> 35 -> 37 -> 88 -> 55 -> 31 -> 83 -> 24 -> 44 -> 10 -> 57 -> 38 -> 48 -> 36 -> 68 -> 84 -> 40 -> 29 -> 20 -> 14 -> 12 -> 97 -> 93 -> 11 -> 30 -> 86 -> 49 -> 92



In [40]:
lat_mean = df_sitios_bog["Lat"].mean()
lng_mean = df_sitios_bog["Lng"].mean()

bogota_map = folium.Map(location=[lat_mean, lng_mean], zoom_start=12, tiles="cartodbdark_matter")
for idx, row in df_sitios_bog.iterrows():
    if idx == "Bavaria SA.":
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=10, color="yellow", fill=True, fill_color="white", fill_opacity=0.6, popup=idx).add_to(bogota_map)
    else:
        folium.CircleMarker([row["Lat"], row["Lng"]], radius=5, color="blue", fill=True, fill_color="white", fill_opacity=0.8, popup=idx).add_to(bogota_map)

for i in range(len(ruta) - 1):
    coordinates = [
        df_sitios_bog.reset_index().loc[ruta[i], ["Lat", "Lng"]].values.tolist(),
        df_sitios_bog.reset_index().loc[ruta[i + 1], ["Lat", "Lng"]].values.tolist(),
    ]

    color = "gray"
    weight = 2

    pl = folium.PolyLine(coordinates, color=color, weight=weight)
    bogota_map.add_child(pl)

    arrows = get_arrows(locations=coordinates, color=color, size=2, n_arrows=1)
    for arrow in arrows:
        arrow.add_to(bogota_map)


bogota_map